# EDA for Nottingham Dataset

## Functions for preprocessing Nottigham dataset

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt 
import os
import re
from collections import Counter

sns.set_style("dark")
global control_var
control_var = True
def get_tuneBooks_file_names(path):
    tuneBook_filenames = []
    
    for tuneBook_name in os.listdir(path):
        if tuneBook_name.endswith('.abc'):
            tuneBook_filenames.append(tuneBook_name)
    
    #return ["jigs.abc"]
    return tuneBook_filenames
def split_abc_song(abc_song):
    k_index = abc_song.find('K:')

    split_index = abc_song.find('\n', k_index)

    header = abc_song[:split_index]
    body = abc_song[split_index+1:]
    return header, body

def tuneBook_to_dataframe(tuneBook):
    song_list = tuneBook.split("\n\n")
    song_list = [song.strip() for song in song_list]
    songs_header_body_format = [split_abc_song(song) for song in song_list if song != '']
    """
    for (h,b) in songs_header_body_format:
        print("header")
        print(h)
        print("*"*10)
        print("body")
        print(b)
        print("\n")
        print("-"*50)
        print("\n")
    """
    
    return pd.DataFrame(songs_header_body_format, columns=['header', 'body'])

def get_attributes_from_song_header(tuneBook_df,tuneBook_name):
    attributes_column = tuneBook_df["header"]
    attributes_list = list(attributes_column)
    
    data = []
    for string in attributes_list:
        song = {}
        for line in string.split("\n"):
            if not line.startswith("%"):
                if ':' in line:
                    key, value = line.split(":", maxsplit=1)
                    song[key.strip()] = value.strip()
        data.append(song)

    df = pd.DataFrame(data)
    descriptive_names = {
                        'X': 'reference_number',
                        'T': 'title',
                        'S': 'source',
                        'M': 'meter',
                        'L': 'unit_note_length',
                        'R': 'rhythm',
                        'P': 'parts',
                        'K': 'key',
                        'F': 'file_name',
                        'N': 'notes'
                        }
    df.rename(columns=descriptive_names, inplace=True)
    df["tuneBook"] = tuneBook_name
    df["original_header"] =  tuneBook_df["header"]
    df["original_body"] =  tuneBook_df["body"]
    
    return df

def get_chord_progression(song_body):
    bars = song_body.split("|")
    chord_progression = "|"
    for bar_content in bars:
        match_chords = r'"[^"]*"'
        chords = re.findall(match_chords, bar_content)
        bar_chords = ''.join(chords)
        if bar_chords != "":
            chord_progression += bar_chords +'|'
    return chord_progression

def get_chord_occurrences(song_body):
    bars = song_body.split("|")
    chord_progression = "|"
    match_chords = r'"[^"]*"'
    chords = [re.findall(match_chords, bar_content)  for bar_content in bars if bar_content]
    chords = [chord for bar_chords in chords for chord in bar_chords]
    chord_occurrences = dict(Counter(chords))
    return chord_occurrences

def erase_double_bars(song):
    pattern = r'\|{2,}'
    return re.sub(pattern,'|',song)

def add_ending_char(song):
    song_with_end = song 
    if not song.endswith(']'):
        if(not song.endswith('|')):
            song_with_end = song_with_end+'|]'
        else:
            pattern = r'\|$'
            song_with_end = re.sub(pattern,'|]',song)
    return song_with_end

def get_chords_data(song_body=""):
    #song_body = 'P:A\nD|"Gm"GB2G|"D7"^Fd2D|"Gm"GB2G|"D7"^FA2D|"Gm"GB2G|"D"^Fg -"Cm"gc|\\\n"Gm/d"B2 "D7"A2|"Gm"G3:|\nP:B\nd|"Gm"dg2d|"Gm"gb2d|"F"cf2c|"F"fa2c|"Eb"Be2d/2c/2|"Gm"dg -"Cm"gc|\\\n"Gm/d"B2 "D7"A2|"Gm"G3:|'
    clean_body_song = ""
    match_pattern = ""
    for line in song_body.splitlines():
        #if not line.startswith("%"):
            #if not "P:" in line:
        if not line.startswith(('P:', 'M:', '%', "L:", 'K:')):
            #print("Header detected!!!",line)
            new_line = line.strip("\\")
            match_pattern = r"[: ]"
            new_line = re.sub(match_pattern, "", new_line)
            clean_body_song = clean_body_song+new_line
            if('||||' in clean_body_song):
                match_pattern = r'\|\|\|\|'
                clean_body_song = re.sub(match_pattern, "|", clean_body_song)
                print("new line",clean_body_song)
                print("old line",line)
                print("-"*10)

    clean_body_song = erase_double_bars(clean_body_song)
    clean_body_song = add_ending_char(clean_body_song)

    if 'K' in clean_body_song:
        print("Found invalid character",clean_body_song)
        raise Exception         
    chord_progression = get_chord_progression(clean_body_song)
    chords_data = get_chord_occurrences(clean_body_song)
    chords_data["chord_progression"] = chord_progression

    chords_data["clean_body"] = clean_body_song
    return chords_data

### treat strings line by line to ignore comments and parts!!!!!
def get_song_characteristics_from_body(tuneBook_df, header_col_name="original_body"):
    songs_bodies=tuneBook_df[header_col_name]
    #global control_var
    #print(songs_bodies)
    #if(control_var):
        #control_var = False
    #df = tuneBook_df.join(songs_bodies.apply(get_chords_data))
    new_data = songs_bodies.apply(get_chords_data)
    chords_df = new_data.apply(pd.Series)
    chords = chords_df['chord_progression']
    chords_df = chords_df.drop('chord_progression', axis=1)
    chords_df["chord_progression"] = chords
    tuneBook_df = tuneBook_df.join(chords_df)
    #print(tuneBook_df.columns)
    #print("*"*20)   
    # get anacrusis bool to see that the dataset is balanced
    #tuneBook["num_notes_per_bar"] # count num -> useless? to measure intensity?
    #tuneBook["num_bars"] # count number of || ins song -> to detect possible outliers?
    #tuneBook["num_notes_in_song"] # count a,b,c,d...in string, match regex "a"   
    #tuneBook["chord_progression"] #string "|"C"|"Dm"|" -> to count chords and to have a "Tree view of common chord progressions"
    #tuneBook["multiple_parts"] #bool -> to treat songs with multiple parts
    ##  Count types of notes in each song to see the proportion of notes for a given key
    ## Maybe by chord also?
    return tuneBook_df
def get_songs_metadata(songs_dataframe, tuneBook_name):
    #songs_dataframe["number_in_tunebook"] = get_attributes_from_song_header(songs_dataframe)
    df = get_attributes_from_song_header(songs_dataframe,tuneBook_name)
    df = get_song_characteristics_from_body(df)
    return df

def get_chord_columns_as_occurrences(songs_df):
    chords_names = songs_df.columns[songs_df.columns.str.contains('"[^"]*"')]
    chords_occurrences = songs_df[chords_names].sum()
    return chords_occurrences



def drop_rows_by_chord_occurrence(songs_df,drop_threshold=15):
    
    chords_occurrences = get_chord_columns_as_occurrences(songs_df)
    ## Gets the names of the chords that appear less times than the threshold.
    chords_names_least_occurrences = chords_occurrences[chords_occurrences<drop_threshold].index.to_list()
    print("number chords least occurrences: ",len(chords_names_least_occurrences))
    print("chords least occurrences",chords_names_least_occurrences)

    ## Applies a logical OR to the selected columns in order to drop the rows that have a value in the corresponding column
    mask_drop_songs_with_low_chords_sample = songs_df[chords_names_least_occurrences].apply(lambda row: row>0).any(axis=1)
    drop_num = mask_drop_songs_with_low_chords_sample.value_counts()
    print("number of values to be dropped:\n",drop_num)

    ##Apply the negative mask to preserve the rows that don't have occurrences of the selected chords
    clean_songs_df = songs_df[~mask_drop_songs_with_low_chords_sample]

    ##Drop columns of useless chords
    clean_songs_df = clean_songs_df.drop(chords_names_least_occurrences, axis=1)
    clean_songs_df = clean_songs_df.reset_index(drop=True)

    return clean_songs_df

def drop_rows_by_extended_chords(songs_df,chords_to_drop=[]):
    
    chords_occurrences = get_chord_columns_as_occurrences(songs_df)

    ## Gets the names of the chords that have '/' 
    extended_chords_names = songs_df.columns[songs_df.columns.str.contains("/")].to_list()
    extended_chords_names = extended_chords_names + chords_to_drop

    ## Applies a logical OR to the selected columns in order to drop the rows that have a value in the corresponding column
    mask_drop_songs_with_extended_chords = songs_df[extended_chords_names].apply(lambda row: row>0).any(axis=1)
    drop_num = mask_drop_songs_with_extended_chords.value_counts()
    print("number of values to be dropped:\n",drop_num)

    ##Apply the negative mask to preserve the rows that don't have occurrences of the selected chords
    clean_songs_df = songs_df[~mask_drop_songs_with_extended_chords]

    ##Drop columns of useless chords
    clean_songs_df = clean_songs_df.drop(extended_chords_names, axis=1)
    clean_songs_df = clean_songs_df.reset_index(drop=True)

    return clean_songs_df


def count_bars_in_songs(songs_df):
    return 0

def calculate_note_length(songs_df_row,col_name="unit_note_length"):
    new_note_length = ""
    ## See: https://abcnotation.com/wiki/abc:standard:v2.1#lunit_note_length 
    map_meter_to_note_length = {
        "4/4":"1/8",
        "6/8":"1/8",
        "3/4":"1/8",
        "2/4":"1/16",
        "9/8":"1/8",
        "2/2":"1/8",
        "3/2":"1/8",
        "6/4":"1/8"
    }
    if(pd.isna(songs_df_row[col_name])):
        print("NA detected!")
        meter = songs_df_row["meter"]
        new_note_length = map_meter_to_note_length[meter]
    else:
        new_note_length = songs_df_row[col_name]

    return new_note_length

def fill_missing_note_length(songs_df):
    col_with_na = "unit_note_length"
    songs_df[col_with_na] = songs_df.apply(calculate_note_length,axis=1)
    return songs_df

def drop_useless_columns(df,drop_columns=["Y","notes","parts","rhythm","source"]):
    clean_df = df
    df_columns = clean_df.columns
    for useless_column in drop_columns:
        if(useless_column in df_columns):
            clean_df = clean_df.drop(useless_column, axis=1)
    return clean_df


def generate_clean_header(songs_df,for_training=False):
    descriptive_name_to_code = {
                        'reference_number':'X',
                        'title':'T',
                        'source':'S',
                        'meter':'M',
                        'unit_note_length':'L',
                        'rhythm':'R',
                        'parts':'P',
                        'key':'K',
                        'file_name':'F',
                        'notes':'N',
                        }
    header_cols = []
    if for_training:
        header_cols = ["meter","unit_note_length","key"]
    else:
        header_cols = ["reference_number","title","meter","unit_note_length","key"]

    compose_header_codes_and_values = lambda row: [f'{descriptive_name_to_code[col_name]}:{row[col_name]}' for col_name in header_cols]
    songs_df["clean_header"] = songs_df[header_cols].apply(lambda row: ('\n'.join(compose_header_codes_and_values(row))), axis=1)
    if for_training:
        append_chord_prog = lambda row: f'{row["clean_header"]}\n{row["chord_progression"]}\n'
        songs_df["clean_header"] = songs_df[["clean_header","chord_progression"]].apply(append_chord_prog, axis=1)

    return songs_df

def generate_clean_songs(songs_df,for_training=False):
    clean_songs_df = generate_clean_header(songs_df,for_training)
    additional_line =  "\n" if not for_training else ""
    append_body_to_header = lambda row: f'{row["clean_header"]}{additional_line}{row["clean_body"]}\n'
    clean_songs_df["clean_song"] = clean_songs_df.apply(append_body_to_header, axis=1)
    return clean_songs_df

def prepare_dataset_for_EDA(relative_path="",drop_by_occurrences=True, drop_by_extended_chord=True, min_chord_progression_length=20):
    absolute_path = os.getcwd()
    #relative_path = "notebooks/data/NottinghamData/nottingham_database"
    #relative_path = "notebooks/data/NottinghamCleaned/nottingham_match/python/data/nottingham_jukedeck/ABC_cleaned"
    absolute_path = os.path.join(absolute_path, relative_path)
    songs_df = pd.DataFrame()
    list_tuneBooks = get_tuneBooks_file_names(absolute_path)
    for abc_tuneBook_filename in list_tuneBooks:
        file_path = os.path.join(absolute_path, abc_tuneBook_filename)
        
        with open(file_path) as tuneBook:
            contents = tuneBook.read()
            df = tuneBook_to_dataframe(contents)
            df1 = get_songs_metadata(df,abc_tuneBook_filename)
            songs_df = pd.concat([songs_df, df1], ignore_index=True)

    ## Sets chords columns as int types
    chords_names = songs_df.columns[songs_df.columns.str.contains('"[^"]*"')]
    songs_df[chords_names] = songs_df[chords_names].fillna(0).astype(int)

    

    chords_occurrences = songs_df[chords_names].sum().sort_values(ascending=True)

    #chords_occurrences = chords_occurrences.reset_index(drop=True)
    chords_occurrences = chords_occurrences.reset_index()
    chords_occurrences_before_drop = chords_occurrences.rename({0:"count","index":"chords"},axis=1)
    
    clean_songs_df = songs_df
    clean_songs_df = drop_useless_columns(clean_songs_df)


    if(drop_by_occurrences):
        clean_songs_df = drop_rows_by_chord_occurrence(clean_songs_df)


    if(drop_by_extended_chord):
        clean_songs_df = drop_rows_by_extended_chords(clean_songs_df)

    
    ## Drops rows that don't have the minimum chord progression length
    mask_no_chord_progression = clean_songs_df["chord_progression"].str.len()>min_chord_progression_length
    clean_songs_df = clean_songs_df[mask_no_chord_progression]
    clean_songs_df = clean_songs_df.reset_index(drop=True)

    
    ## X: 102 meter appears after they key so its meter is dropped in the cleaning.
    ## its the only song with this format
    clean_songs_df.loc[clean_songs_df["meter"].isna(),"meter"] = "6/8"
    
    ## Fill empty values of note_length according to the meter of the song
    clean_songs_df = fill_missing_note_length(clean_songs_df)

    #for song in clean_songs_df["chord_progression"]:
    #    print(song)
    #    print("**"*10)
    return songs_df, clean_songs_df, chords_occurrences_before_drop



## EDA Original and Clean Dataset

In [ ]:
original_dataset_relative_path = "notebooks/data/NottinghamCleaned/nottingham_match/python/data/nottingham_jukedeck/ABC_cleaned"
songs_df, clean_songs_df,chords_occurrences = prepare_dataset_for_EDA(relative_path=original_dataset_relative_path)
sns.barplot(data=chords_occurrences,y="count", x='chords',edgecolor='none',color="magenta")

In [ ]:
songs_df["original_body"].str.contains("P:")

In [ ]:
chords_names = clean_songs_df.columns[clean_songs_df.columns.str.contains('"[^"]*"')]

chords_occurrences_df = clean_songs_df[chords_names].sum().sort_values(ascending=False).reset_index()
print(chords_occurrences_df.columns)
sns.barplot(data=chords_occurrences_df,y="index", x=0,edgecolor='none',color="magenta")

In [ ]:
sns.histplot(songs_df["key"].astype("category"))

In [ ]:
#fig, ax = plt.subplots()
ax = sns.histplot(songs_df["key"].astype("category"),label="abc")
ax = sns.histplot(clean_songs_df["key"].astype("category"), ax=ax,label="something 2")
ax.legend()
plt.show()

In [ ]:
clean_songs_df.columns[clean_songs_df.columns.str.contains('"[\(]')]

In [ ]:
chord_names = clean_songs_df.columns[clean_songs_df.columns.str.contains('"')].to_list()
sum_chords = clean_songs_df[chord_names].sum().sort_values().reset_index()
chords = "|"
for chord_name,chord_count in zip(sum_chords["index"],sum_chords[0]):
    chords = chords+chord_name+'|'
    print(chord_name,chord_count)
print(chords)

In [ ]:
print(clean_songs_df["chord_progression"].str.contains("segno").value_counts())

In [ ]:
print(songs_df["chord_progression"].str.contains("segno").value_counts())

In [ ]:
for song,title,book in zip(clean_songs_df["clean_body"],clean_songs_df["title"],clean_songs_df["tuneBook"]):
    print(title," ", book)
    print(song)
    print("**"*10)

In [ ]:
descriptive_names = {
                        'X': 'reference_number',
                        'T': 'title',
                        'S': 'source',
                        'M': 'meter',
                        'L': 'unit_note_length',
                        'R': 'rhythm',
                        'P': 'parts',
                        'K': 'key',
                        'F': 'file_name',
                        'N': 'notes'
                        }

In [ ]:
category_name = "meter"
category_value_counts = clean_songs_df[category_name].astype("category").value_counts()
print(category_value_counts)
print("\nSum of values:",category_value_counts.sum())
print("\nNaN values:",clean_songs_df[category_name].isna().sum())
sns.histplot(clean_songs_df[category_name].astype("category"))

In [ ]:
category_name = "rhythm"
category_value_counts = songs_df[category_name].astype("category").value_counts()
print(category_value_counts)
print("\nSum of values:",category_value_counts.sum())
print("\nNaN values:",songs_df[category_name].isna().sum())
sns.histplot(songs_df[category_name].astype("category"))

In [ ]:
category_name = "unit_note_length"
category_value_counts = clean_songs_df[category_name].astype("category").value_counts()
print(category_value_counts)
print("\nSum of values:",category_value_counts.sum())
print("\nNaN values:",clean_songs_df[category_name].isna().sum())
sns.histplot(clean_songs_df[category_name].astype("category"))

In [ ]:
category_name = "key"
category_value_counts = clean_songs_df[category_name].astype("category").value_counts()
print(category_value_counts)
print("\nSum of values:",category_value_counts.sum())
print("\nNaN values:",clean_songs_df[category_name].isna().sum())
sns.histplot(clean_songs_df[category_name].astype("category"))

In [ ]:
category_name = "tuneBook"
category_value_counts = clean_songs_df[category_name].astype("category").value_counts()
print(category_value_counts)
print("\nSum of values:",category_value_counts.sum())
print("\nNaN values:",clean_songs_df[category_name].isna().sum())
rename_dict = {category_name:'count','index':"tuneBook"}
aux_df_1 = clean_songs_df[category_name].astype("category").value_counts().reset_index().rename(rename_dict,axis=1)
aux_df_2 = songs_df[category_name].astype("category").value_counts().reset_index().rename(rename_dict,axis=1)
ax1= sns.barplot(data=aux_df_1, y='tuneBook', x='count',label="Clean Dataset", color='#8D34DC',alpha=0.9)
sns.barplot(ax=ax1,data=aux_df_2, y='tuneBook', x='count',label="Original Dataset", color='magenta',alpha=0.4)
plt.title("Original vs Preprocessed Dataset")
plt.legend(loc='lower right')
plt.show()

In [ ]:
category_name = "unit_note_length"
category_value_counts = clean_songs_df[category_name].astype("category").value_counts()
print(category_value_counts)
print("\nSum of values:",category_value_counts.sum())
print("\nNaN values:",clean_songs_df[category_name].isna().sum())
rename_dict = {category_name:'count','index':"unit_note_length"}
aux_df_1 = clean_songs_df[category_name].astype("category").value_counts().reset_index().rename(rename_dict,axis=1)
aux_df_2 = songs_df[category_name].astype("category").value_counts().reset_index().rename(rename_dict,axis=1)
fig, (ax1, ax2) = plt.subplots(1, 2 ,sharey=True)
sns.barplot(ax=ax2,data=aux_df_1, x='unit_note_length', y='count',label="Clean Dataset", color='#8D34DC',alpha=0.9)
sns.barplot(ax=ax1,data=aux_df_2, x='unit_note_length', y='count',label="Original Dataset", color='magenta',alpha=0.4)
fig.suptitle('Original vs Preprocessed Dataset')
fig.legend()
plt.show()

In [ ]:
df_aux = pd.DataFrame({
        "clean dataset":[clean_songs_df.shape[0]],
        "original dataset":[songs_df.shape[0]]
        })
df_aux["dropped values"] = df_aux["original dataset"]-df_aux["clean dataset"]
ordered_list_values = ["dropped values","original dataset","clean dataset"]
magenta_palette = sns.dark_palette('magenta', n_colors=len(ordered_list_values))
reversed_magenta_palette = list(reversed(magenta_palette))
color_dict  = dict(zip(ordered_list_values, reversed_magenta_palette))
ordered_list_values = ["original dataset","dropped values","clean dataset"]
df_aux = df_aux[ordered_list_values]
df_aux = df_aux.T.reset_index().rename({"index":"values",0:"count"},axis=1)
ax = sns.barplot(data=df_aux,x="values",y="count",palette=magenta_palette,alpha=0.5)
plt.title("Values in Dataset")


In [ ]:
clean_songs_df["meter"].value_counts()

In [ ]:
clean_songs_df["meter"].isna().sum()

In [ ]:
print(clean_songs_df["unit_note_length"].value_counts())
print(clean_songs_df["unit_note_length"].isna().sum())

In [ ]:
clean_songs_df.columns.sort_values()

In [ ]:
print("Number of NaN in the entire dataset:",clean_songs_df.isna().sum().sum())

## Save Intermediate datasets

In [ ]:
## Print clean dataset for transposing

#for song in generate_clean_songs(clean_songs_df,for_training=False)["clean_song"]:
#    print(song)
def save_dataframe(df,relative_path,file_name):
    reorder_columns = df.columns.sort_values(ascending=False)
    df_to_be_saved = df[reorder_columns]
    print(file_name," dataset shape",df_to_be_saved.shape)
    print(df_to_be_saved.columns)
    print("-"*10)
    df_to_be_saved.to_pickle(f'{relative_path}/{file_name}.pkl')

def load_dataframe(relative_path,dataframe_name):
    df = pd.read_pickle(f'{relative_path}/{dataframe_name}.pkl')    
    return df

def write_tuneBook_file(df,relative_path,file_name):
    with open(f'{relative_path}/{file_name}.abc', 'w') as f:
        for song in df["clean_song"]:
            f.write(song+'\n')
relative_path ="notebooks/data/original_dataset"
clean_dataset_for_augmentation_df = generate_clean_songs(clean_songs_df,for_training=False)
filename_name = 'clean_original_data'
save_dataframe(clean_dataset_for_augmentation_df,relative_path,filename_name)
write_tuneBook_file(clean_dataset_for_augmentation_df,relative_path,filename_name)

clean_dataset_for_training_df = generate_clean_songs(clean_songs_df,for_training=True)
filename_name = 'clean_original_training_data'
save_dataframe(clean_dataset_for_training_df,relative_path,filename_name)



In [ ]:
relative_path ="notebooks/data/original_dataset"
filename_name = 'clean_original_data'
clean_dataset_for_augmentation_df = load_dataframe(relative_path,filename_name)
filename_name = 'clean_original_training_data'
clean_dataset_for_training_df = load_dataframe(relative_path,filename_name)

In [ ]:
print("augmentation dataset shape",clean_dataset_for_augmentation_df.shape)
print(clean_dataset_for_augmentation_df.columns)
print("training dataset shape",clean_dataset_for_training_df.shape)
print(clean_dataset_for_training_df.columns)

In [ ]:
for song in clean_dataset_for_augmentation_df.head()["clean_song"]:
    print(song)

In [ ]:
for song in clean_dataset_for_training_df.head()["clean_song"]:
    print(song)

## Load Augmented Datasets

In [ ]:

augmented_dataset_relative_path = "notebooks/data/augmented_dataset"
_, clean_songs_augmented_df ,chords_occurrences_aug= prepare_dataset_for_EDA(relative_path=augmented_dataset_relative_path,drop_by_occurrences=True, drop_by_extended_chord=True)
clean_songs_augmented_df


## EDA Augmented datasets

In [ ]:
 def get_df_with_key_count(df,col_name="key"):
    category_name = col_name
    key_categories = df[category_name].astype("category").value_counts()
    key_categories = key_categories.reset_index().sort_values(by=[category_name,"index"],ascending=[False, True])
    #key_categories["tuneBook"] = clean_songs_augmented_df["tuneBook"]
    key_categories = key_categories.rename(columns={'index': category_name, category_name: 'count'})

    major_or_minor = lambda row: "Minor" if 'm' in row[category_name] else "Major"
    key_categories["mode"] = key_categories.apply(major_or_minor,axis=1)
    print(key_categories.columns)
    return key_categories

key_categories_1 = get_df_with_key_count(clean_songs_augmented_df)
key_categories_1["dataset_type"] = "augmented"
#sns.catplot(data=key_categories_1,y="key",x="count",order=key_categories_1['key'],kind='bar')
key_categories_2 = get_df_with_key_count(clean_songs_df)
key_categories_2["dataset_type"] = "original"
#sns.catplot(data=key_categories_2,y="key",x="count",order=key_categories_2['key'],kind='bar')

key_categories = pd.concat([key_categories_1,key_categories_2],axis=0)
key_categories['dataset_type'] = pd.Categorical(key_categories['dataset_type'], categories=['original', 'augmented'], ordered=True)
key_categories['count'] = pd.to_numeric(key_categories['count'])
print(key_categories.dtypes)
key_categories['key'] = key_categories['key'].astype('category')
key_categories['mode'] = key_categories['mode'].astype('category')
print(key_categories.dtypes)
magenta_palette = sns.light_palette('magenta', n_colors=len(key_categories['count'].unique()))
color_dict  = dict(zip(key_categories['count'].sort_values(ascending=True).unique(), magenta_palette))
ax = sns.catplot(data=key_categories,y="key",x="count",kind='bar',col='dataset_type')
ax.fig.suptitle('Key Count Comparison',y=1.05)

In [ ]:
ax1 = sns.barplot(label="Augmented Dataset",data=key_categories_2,y="key",x="count",order=key_categories_2['key'], color='#8D34DC',alpha=0.9)
ax1 = sns.barplot(ax=ax1,label="Original Dataset",data=key_categories_1,y="key",x="count",order=key_categories_1['key'], color='magenta',alpha=0.4)
ax1.set_title("Key Count Comparison")
ax1.legend()
plt.show()

In [ ]:
def get_key_bar_plots(old_df, new_df):
    chords_occurrences_aug = old_df
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 8),sharex=True)
    sns.barplot(ax=ax1,data=chords_occurrences_aug.sort_values(by="count",ascending=False),x="count", y='chords',edgecolor='none',color="magenta",alpha=0.5)
    ax1.set_title('Before Preprocessing')

    clean_songs_augmented_df= new_df
    chords_names_aug = clean_songs_augmented_df.columns[clean_songs_augmented_df.columns.str.contains('"[^"]*"')]

    chords_aug_occurrences_df = clean_songs_augmented_df[chords_names_aug].sum().sort_values(ascending=False).reset_index()
    print(chords_aug_occurrences_df.columns)
    column_names = {
        'index': 'chords',
        0:'count'
        }
    chords_aug_occurrences_df = chords_aug_occurrences_df.rename(columns=column_names)
    sns.barplot(ax=ax2,data=chords_aug_occurrences_df,y="chords", x="count",edgecolor='none',color="magenta",alpha=0.5)
    ax2.set_title('After Preprocessing')

    fig.suptitle('Chord Count in Dataset', y=0.94)

get_key_bar_plots(chords_occurrences_aug,clean_songs_augmented_df)  

## Save Augmented Dataset

In [ ]:
#clean_dataset_for_augmentation_df = generate_clean_songs(clean_songs_augmented_df,for_training=False)
print("dataset shape",clean_songs_augmented_df.shape)
print("dataset\n",clean_songs_augmented_df.columns.sort_values(ascending=False))
clean_augmented_dataset_for_training_df = generate_clean_songs(clean_songs_augmented_df,for_training=True)
relative_path ="notebooks/data/augmented_dataset"
filename_name = 'clean_augmented_data'
save_dataframe(df=clean_augmented_dataset_for_training_df,file_name=filename_name,relative_path=relative_path)

In [ ]:
relative_path ="notebooks/data/augmented_dataset"
filename_name = 'clean_augmented_data'
clean_augmented_dataset_for_training_df = pd.read_pickle(f'{relative_path}/{filename_name}.pkl')
clean_augmented_dataset_for_training_df.columns

In [ ]:
def count_double_bars(song):
    pattern = r'\|{2,}'
    matches = re.findall(pattern, song)
    count = len(matches)
    return count
    


s = erase_double_bars('one |||| two || three |a|a| four|||')
add_ending_char(s)

In [ ]:
mask = clean_augmented_dataset_for_training_df['tuneBook'] == 'dataset_min4.abc'
sample_df = clean_augmented_dataset_for_training_df[mask]
ends = 0
in_middle = 0
no_end = 0
weird = 0
regex_bars_found = 0
for song in sample_df['clean_body']:
    regex_bars_found += count_double_bars(song)
    print(song)
    if '||' in song:
        if song.endswith('||'):
            ends +=1
        else:
            in_middle += 1
    else:
        no_end += 1
    if ']' in song:
        no_end += 1
print("ends",ends)
print("in middle",in_middle)
print("no end",no_end)
print("total",ends+in_middle+no_end)
print("regex bars found", regex_bars_found)
print("] char in song", weird)


In [ ]:
for song in sample_df['clean_body'].apply(erase_double_bars):
    print(song)

In [ ]:
for song in clean_songs_df["clean_body"]:
    print(song)

In [ ]:
song = '(3ABc|"D"d2f2d2|fg|"D"afafd2ga|||"G"bgbg"A7"e2fg||"D"dcdAFAdf||"Em"edcB"A7"A2g2|||||"D"fgaf"A7"bgec|"D"d2f2d2|]'
song = erase_double_bars(song)
song = add_ending_char(song)
song